In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Load and preprocess the data
data = pd.read_csv('history.csv')
data['TimeStamp'] = pd.to_datetime(data['TimeStamp'])
data.set_index('TimeStamp', inplace=True)

# Scale the Temperature data
scaler_temp = MinMaxScaler(feature_range=(0, 1))
data['Temperature'] = scaler_temp.fit_transform(data[['Temperature']])

# Create sequences for LSTM with 2-hour future predictions
def create_sequences(data, seq_length, future_steps):
    X, y = [], []
    for i in range(len(data) - seq_length - future_steps + 1):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length:i + seq_length + future_steps].flatten())  # Get the next 2 hours
    return np.array(X), np.array(y)

temperature_data = data[['Temperature']].values
seq_length = 24 # Use past 24 hours of data
future_steps = 2  # Predict the next 2 hours
X, y = create_sequences(temperature_data, seq_length, future_steps)

# Split into train and test sets
split_ratio = 0.8
split_index = int(len(X) * split_ratio)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Build and train the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(50),
    Dense(future_steps)  # Output layer with 2 outputs (next 2 hours)
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=35, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Predict temperature for the next 2 hours on the test set and inverse transform
predicted_temp_scaled = model.predict(X_test)
predicted_temp = scaler_temp.inverse_transform(predicted_temp_scaled)
y_test_actual = scaler_temp.inverse_transform(y_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test_actual, predicted_temp))
mae = mean_absolute_error(y_test_actual, predicted_temp)
r2 = r2_score(y_test_actual, predicted_temp)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

# Plot actual vs predicted temperatures for the next 2 hours
plt.figure(figsize=(12, 6))
plt.plot(y_test_actual[:50, 0], label='Actual Temperature (1st hour ahead)', color='blue')
plt.plot(predicted_temp[:50, 0], label='Predicted Temperature (1st hour ahead)', linestyle='--', color='blue')
plt.xlabel("Time")
plt.ylabel("Temperature")
plt.title("Actual vs Predicted Temperature (Next 2 Hours)")
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(y_test_actual[:50, 1], label='Actual Temperature (2nd hour ahead)', color='orange')
plt.plot(predicted_temp[:50, 1], label='Predicted Temperature (2nd hour ahead)', linestyle='--', color='orange')
plt.xlabel("Time")
plt.ylabel("Temperature")
plt.title("Actual vs Predicted Temperature (Next 2 Hours)")
plt.legend()
plt.show()

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Save the model
model.save("tem_predict_next_2_hours.keras")


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

def predict_temperature():
    # Load model and scaler
    model = load_model("tem_predict_next_2_hours.keras")
    scaler_temp = MinMaxScaler(feature_range=(0, 1))

    # Load data and preprocess
    data = pd.read_csv('history.csv')
    data['TimeStamp'] = pd.to_datetime(data['TimeStamp'])
    data.set_index('TimeStamp', inplace=True)
    data['Temperature'] = scaler_temp.fit_transform(data[['Temperature']])

    # Prepare latest sequence (last 24 hours) for prediction
    latest_sequence = data[['Temperature']].values[-24:]
    latest_sequence = latest_sequence.reshape(1, 24, 1)  # Reshape for model input

    # Predict next 2 hours and inverse transform
    predicted_temp_scaled = model.predict(latest_sequence)
    predicted_temp = scaler_temp.inverse_transform(predicted_temp_scaled)

    return predicted_temp[0][0], predicted_temp[0][1]

print(predict_temperature())